In [1]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports
import GEMS_TCO
from GEMS_TCO import kernels
from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels_nov25 
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2



In [44]:
from pathlib import Path
# Assuming 'config' and 'load_data' class are defined and imported elsewhere

# --- Parameters derived from your framework ---
v: float = 0.5
space: List[str] = ['4', '4']
days: List[str] = ['0', '31']
mm_cond_number: int = 20
# --- End of framework parameters ---

lat_lon_resolution = [int(s) for s in space]
days_s_e = [int(d) for d in days]
days_list = list(range(days_s_e[0], days_s_e[1]))

# These values were not in the framework, so they remain as set in your snippet
years = ['2024']
month_range = [7] 

# Assuming 'config' is available in your environment
output_path = input_path = Path(config.mac_estimates_day_path)

## load ozone data from amarel
data_load_instance = load_data2(config.mac_data_load_path)

# Call the function using the variables from the framework
df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      # <-- Add this
lon_range=[123.0, 133.0]   # <-- Add this
)

Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


## fit 8 hours of data idx_for_datamap = [i*8, (i+1)*8]

## fit one hour idx_for_datamap = [i*8, (i*8+1)]

In [40]:
# List to hold the aggregated tensor for each day
daily_aggregated_tensors = [] 

# List to hold the map (dict) of hourly data for each day
daily_hourly_maps = []        

for day_index in range(31):
    # Calculate the start and end indices for the 8 hours of data
    hour_start_index = day_index * 8
    #hour_end_index = (day_index + 1) * 8
    hour_end_index = day_index * 8 + 1
    hour_indices = [hour_start_index, hour_end_index]
    
    # Load the data for the current day
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
        df_map, 
        hour_indices, 
        ord_mm=None,  
        dtype=torch.float 
    )
    
    # Append the day's data to their respective lists
    daily_aggregated_tensors.append(day_aggregated_tensor)
    daily_hourly_maps.append(day_hourly_map) 

print(daily_aggregated_tensors[0].shape)
#print(daily_hourly_maps[0])

torch.Size([1120, 4])


In [41]:
instance1 = kernels.vecchia_experiment(0.5, daily_hourly_maps[0], daily_aggregated_tensors[0], nns_map, mm_cond_number, nheads=10)

a = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769]

early_stop_vecc_5000= [2.8770e+01, 9.6e-01, 1.08e+00, 6.6789e-07, 1.13e-06, 00,1.91e+00]
scheduler_vecc_5000= [30.823, 1.325, 2.30, 5.191e-06, -9.755e-07, 0.0, 3.17]

a = early_stop_vecc_5000 #  2174.9120
a = [2.9527e+01, 1.3223e+00, 2.2772e+00, 1.3499e-05, 5.9777e-06, 0.0000e+00,
        3.2343e+00] # 2108.4802


params = torch.tensor(a, dtype=torch.float64, requires_grad=True)
cov_map = instance1.cov_structure_saver(params, instance1.matern_cov_anisotropy_v05)  
instance1.vecchia_oct22( params, instance1.matern_cov_anisotropy_v05, cov_map )


tensor(2108.4801, dtype=torch.float64, grad_fn=<AddBackward0>)

In [42]:
v = 0.5 # smooth
mm_cond_number = 20
nheads = 300
#nheads = 1230
lr = 0.01
step = 80
gamma_par = 0.5
epochs = 300


In [15]:
params

tensor([2.9527e+01, 1.3223e+00, 2.2772e+00, 1.3499e-05, 5.9777e-06, 0.0000e+00,
        3.2343e+00], dtype=torch.float64, requires_grad=True)

In [32]:
day_indices = [0] 
for day_idx in day_indices:  

    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]


    # Initial parameters (full vector 'a' is for reference)
    a = [27.1033, 0.7379, 1.7117, 0,0,0, 0.6203]

    # NEW: Define params as a list of 1-element tensors (one per parameter)
    params_list = [
        torch.tensor([val], dtype=torch.float64, requires_grad=True) for val in a
    ]

    model_instance = kernels_nov25.fit_vecchia_adams(
            smooth = v,
            input_map = daily_hourly_map,        # 💥 FIX: Use correct loop variable
            aggregated_data = daily_aggregated_tensor, # 💥 FIX: Use correct loop variable
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )
    
    bb= model_instance.full_likelihood( params_list, daily_aggregated_tensor,daily_aggregated_tensor[:,2], model_instance.matern_cov_anisotropy_v05)
    print(bb)

tensor(1861.0600, dtype=torch.float64, grad_fn=<MulBackward0>)


# fit adams

In [43]:
day_indices = [0] 
for day_idx in day_indices:  

    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]


    # Initial parameters (full vector 'a' is for reference)
    a = [27.1033, 0.7379, 1.7117, 0,0,0, 0.6203]

    # NEW: Define params as a list of 1-element tensors (one per parameter)
    params_list = [
        torch.tensor([val], dtype=torch.float64, requires_grad=True) for val in a
    ]

    # NEW: Define learning rates and groups
    lr_slow, lr_fast = 0.005, 0.02 # Assuming these are defined elsewhere
    slow_indices = [ 3,4,5] # e.g., ranges, advection, beta, nugget
    fast_indices = [0,6,1,2] # e.g., sigmasq
    
    # Define Parameter Groups for the optimizer
    param_groups = [
        {'params': [params_list[idx] for idx in slow_indices], 'lr': lr_slow, 'name': 'slow_group'},
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
            
    # --- Data loading is now done *before* the loop ---

    # Define device
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels_nov25.fit_vecchia_adams(
            smooth = v,
            input_map = daily_hourly_map,        # 💥 FIX: Use correct loop variable
            aggregated_data = daily_aggregated_tensor, # 💥 FIX: Use correct loop variable
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # Adjusted optimizer call: Passes the list of parameter groups
    optimizer, scheduler = model_instance.set_optimizer(
            param_groups,     
            lr=lr,            
            betas=(0.9, 0.99), 
            eps=1e-8, 
            step_size=step, 
            gamma=gamma_par
        )

    out, epoch_ran = model_instance.fit_vecc_scheduler_oct23(
            params_list,     # <--- Pass the list of parameter tensors
            optimizer,
            scheduler, 
            model_instance.matern_cov_anisotropy_v05, 
            epochs=epochs
        )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day_idx+1} final results: {out}")


--- Starting Day 1 (2024-07-1) ---
Data size per day: 1250.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [tensor([27.1033], dtype=torch.float64, requires_grad=True), tensor([0.7379], dtype=torch.float64, requires_grad=True), tensor([1.7117], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.6203], dtype=torch.float64, requires_grad=True)]
--- Epoch 1 / Loss: 1961.340872 ---
  Param 0: Value=27.1033, Grad=1.8633586776978561
  Param 1: Value=0.7379, Grad=-10.733745573011923
  Param 2: Value=1.7117, Grad=37.713100738653516
  Param 3: Value=0.0000, Grad=-9.553885460533706e-13
  Param 4: Value=0.0000, Grad=-6.2450045135165055e-15
  Param 5: Value=0.0000, Grad=0.0
  Param 6: Value=0.6203, Grad=-13.710450344960186
  Max Abs Grad: 3.771310e+01
------------------------------
--- Epoch 11 / Loss: 1952.819204 ---
  Param

# fit L-BFGS

In [37]:
day_indices = [0] 
for day_idx in day_indices:  

    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

    # Initial parameters (full vector 'a' is for reference)
    a = [30, 0.66, 0.7, 0,0, 0, 1.5]


    # NEW: Define params as a list of 1-element tensors (one per parameter)
    params_list = [
        torch.tensor([val], dtype=torch.float64, requires_grad=True) for val in a
    ]

    # --- 💥 L-BFGS FIX 💥 ---
    # The L-BFGS optimizer does NOT support parameter groups
    # (e.g., 'slow_group', 'fast_group'). We must pass the
    # flat 'params_list' directly to the optimizer.
    
    # (Removed lr_slow, lr_fast, slow_indices, fast_indices)
    # (Removed param_groups definition)
    # --- END FIX ---
    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
            
    # --- Data loading is now done *before* the loop ---

    # Define device
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels_nov25.fit_vecchia_lbfgs(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # Adjusted optimizer call: Passes the flat list of parameter tensors
    optimizer = model_instance.set_optimizer(
            params_list,       # 💥 FIX: Pass the flat list of tensors
            lr=1.0,            # Standard initial step length for L-BFGS
            max_iter=10,       # Line search iterations per step
            tolerance_grad=1e-4
        )
    
    max_lbfgs_steps = 100
    out, steps_ran = model_instance.fit_vecc_lbfgs(
            params_list,
            optimizer,
            model_instance.matern_cov_anisotropy_v05, # Pass the *actual* method
            max_steps=max_lbfgs_steps
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    # 💥 FIX: Use 'steps_ran' for the print message, not 'epoch_ran'
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {steps_ran+1} steps.")
    print(f"Day {day_idx+1} final results: {out}")


--- Starting Day 1 (2024-07-1) ---
Data size per day: 5000.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [tensor([30.], dtype=torch.float64, requires_grad=True), tensor([0.6600], dtype=torch.float64, requires_grad=True), tensor([0.7000], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([1.5000], dtype=torch.float64, requires_grad=True)]
--- Starting L-BFGS Optimization ---
--- Step 1/100 / Loss: 8147.471414 ---
  Param 0: Value=20.4028, Grad=3.222488216223327
  Param 1: Value=6.6228, Grad=6.571895806238666
  Param 2: Value=2.7459, Grad=6.942704576642983
  Param 3: Value=-0.0000, Grad=-1.390901283038204e-13
  Param 4: Value=0.0000, Grad=-1.42559575255774e-13
  Param 5: Value=0.0000, Grad=0.0
  Param 6: Value=5.7234, Grad=-12.480498270977808
  Max Abs Grad: 1.248050e+01
------------------------------
--- Step 2/10

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

# fit one hour but fix time relevant parameters

In [6]:
# ------------------------------------
# Main Optimization Loop (Updated)
# ------------------------------------

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
df_day_aggregated_list = []
df_day_map_list = []
num_days_to_load = 31 # From reference code

for i in range(num_days_to_load):
    idx_for_datamap = [i*8, i*8+1]
    # Using the new load function from the reference code
    cur_map, cur_df = data_load_instance.load_working_data(
        df_map, 
        idx_for_datamap, 
        ord_mm=None,
        # Using float64 to ensure compatibility with kernels
        dtype=torch.float64 
    )
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

print(f"Data loaded for {len(df_day_map_list)} days.")

# --- 2. Run optimization loop over pre-loaded data ---

# This list is now just for iterating
#days_list = range(len(df_day_map_list)) 
days_list = [0]
for day in days_list:  
    
    # Get the pre-loaded data for this day
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]

    # **CHANGE**: Use a single initialization vector 'a' with fixed indices (3, 4, 5) set to 0.
    # a = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769] # Original
    a = [29.75, 0.98, 1.06, 0, 0, 0, 1.890]
    params = torch.tensor(a, dtype=torch.float64, requires_grad=True)
    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params.detach().numpy()}')
            
    # --- Data loading is now done *before* the loop ---

    # We need to define the device (though we aren't passing it anymore)
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
            # device = device_str  <--- REMOVED: This was causing the TypeError
        )

    start_time = time.time()
    
    # Adjusted optimizer call based on expected return values (step size changed to step)
    lr=0.03
    optimizer, scheduler = model_instance.optimizer_fun_scheduler_same_lr(
        params, 
        lr=lr, 
        betas=(0.9, 0.8), 
        eps=1e-8, 
        step_size=step, # Using the 'step' variable here
        gamma=gamma_par  # Using gamma_par
    ) 

    # --- CRITICAL CORRECTION: Call the new masked function ---
    out, epoch_ran = model_instance.run_vecc_scheduler_same_lr_fit_hour( # <-- Function name changed
        params, 
        optimizer,
        scheduler, 
        model_instance.matern_cov_anisotropy_v05, 
        epochs=epochs
    )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Day {day+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day+1} final results: {out}")

print("\n--- All Days Processed ---")

Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 5000.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [29.75  0.98  1.06  0.    0.    0.    1.89]
Epoch 1, Gradients (Full): [ -0.02896164  10.53635213 -13.62827347   0.           0.
   0.          -2.77344428]
 Loss: 8570.70743603638, Parameters (Full): [29.75  0.98  1.06  0.    0.    0.    1.89]
Epoch 51, Gradients (Full): [-0.07348001 -0.39344868 -0.09763837  0.          0.          0.
  0.83158837]
 Loss: 8315.909261744184, Parameters (Full): [29.83286556  1.31416734  2.30829604  0.          0.          0.
  3.3709474 ]
Epoch 101, Gradients (Full): [-0.08185817 -0.02543247 -0.01331501  0.          0.          0.
 -0.08306768]
 Loss: 8314.451279317555, Parameters (Full): [31.35687947  1.32904135  2.31022858  0.          0.          0.
  3.1419399 ]
Epoch 151, Gradients (Full): [-0.07862879 -0.0057642  -0.00551532  0.          0.          0.
 -0.01512818]
 Loss: 83